In [8]:
# imports
import bz2
import pickle5 as pickle
import pandas as pd

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
# functions
# Load any compressed pickle file
def decompress_pickle(file):
    data = bz2.BZ2File(file, 'rb')
    data = pickle.load(data)
    return data

In [41]:
df = decompress_pickle('6nutrients.pbz2')

df['difference'] = df.apply(lambda x: x['total.calories'] - x['goal.calories'], axis=1)

In [42]:
# remove rows with only one day worth of data
#df1 = df.groupby('id').filter(lambda x : len(x)>1)

#df1['success'] = df1.apply(lambda x: 0 if (x['difference'] < 0 ) else 1, axis=1)

# add shiftid column
df.insert(2, 'shiftid', df.shift(periods=-1, axis=0)['id'])

#df1['new_success'] = df1.apply(lambda x: 0 if (x['id']!=x.shift(periods=1,axis=0)['id']) else 1, axis=1)

In [31]:
#

     FN1    FN2  Final
0  False  False      1
1   True   True      1
2  False  False      1
3   True  False      2
4   True   True      2
5  False  False      1
6   True   True      1
7   True   True      1
